<a href="https://colab.research.google.com/github/LKSfiuza/Machine-Learning-projects/blob/main/Aprendizagem_por_refor%C3%A7o_Q_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import gym
import random

In [19]:
env = gym.make('Taxi-v3').env

In [18]:
env.reset()
env.render()

In [6]:
print(env.action_space)
print(env.observation_space)

Discrete(6)
Discrete(500)


In [7]:
len(env.P)

500

In [8]:
env.P[484]

{0: [(1.0, 484, -1, False)],
 1: [(1.0, 384, -1, False)],
 2: [(1.0, 484, -1, False)],
 3: [(1.0, 464, -1, False)],
 4: [(1.0, 484, -10, False)],
 5: [(1.0, 484, -10, False)]}

Diferença temportal (Q-Learning)

## Qt(s,a) = Qt-1(s,a) + aTDt(a,s)

In [9]:
!pip install numpy==1.23.5


In [10]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])
q_table.shape

(500, 6)

In [11]:
%%time
from IPython.display import clear_output

alpha = 0.1
gamma = 0.6
epsilon = 0.1

for i in range(100000):
  estado = env.reset()
  penalidades, recompensa = 0, 0
  done = False
  while not done:
    # Exploração do ambiente
    if random.uniform(0,1) < epsilon:
      acao = env.action_space.sample()
    else:
      acao = np.argmax(q_table[estado])
    proximo_estado, recompensa, done, info = env.step(acao)
    q_antigo = q_table[estado, acao]
    proximo_maximo = np.max(q_table[proximo_estado])
    q_novo = (1 - alpha) * q_antigo + alpha * (recompensa + gamma * proximo_maximo - q_antigo)
    q_table[estado, acao] = q_novo
    if recompensa == -10:
      penalidades += 1
    estado = proximo_estado

  if i % 100 == 0:
    clear_output(wait=True)
    print(f'Episódio: {i}')

print('Treinamento concluído')


Episódio: 99900
Treinamento concluído
CPU times: user 1min 13s, sys: 10.3 s, total: 1min 23s
Wall time: 1min 22s


In [13]:
env.encode(2, 1, 0, 3)

223

In [14]:
q_table[223]

array([-0.71428401, -0.71426673, -0.71428401, -0.71426673, -5.21428002,
       -5.21428002])

AVALIAÇÃO

In [22]:
total_penalidades = 0
episodios = 50
frames = []

for _ in range(episodios):
  estado = env.reset()
  penalidades, recompensa = 0, 0
  done = False
  while not done:
    acao = np.argmax(q_table[estado])
    estado, recompensa, done, info = env.step(acao)
    if recompensa == -10:
      penalidades += 1
    frames.append({
        'frame': env.render(mode='ansi'),
        'estado': estado,
        'ação': acao,
        'recompensa': recompensa
    })
  total_penalidades += penalidades

print('Episodios', episodios)
print('Penalidades', total_penalidades)

Episodios 50
Penalidades 0


/usr/local/lib/python3.11/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


In [23]:
frames

[{'frame': '+---------+\n|\x1bR\x1b: | : :\x1bG\x1b|\n| : | : : |\n| :\x1b \x1b: : : |\n| | : | : |\n|Y| : |B: |\n+---------+\n  (East)\n',
  'estado': 224,
  'ação': 2,
  'recompensa': -1},
 {'frame': '+---------+\n|\x1bR\x1b: | : :\x1bG\x1b|\n| : | : : |\n| : :\x1b \x1b: : |\n| | : | : |\n|Y| : |B: |\n+---------+\n  (East)\n',
  'estado': 244,
  'ação': 2,
  'recompensa': -1},
 {'frame': '+---------+\n|\x1bR\x1b: | : :\x1bG\x1b|\n| : |\x1b \x1b: : |\n| : : : : |\n| | : | : |\n|Y| : |B: |\n+---------+\n  (North)\n',
  'estado': 144,
  'ação': 1,
  'recompensa': -1},
 {'frame': '+---------+\n|\x1bR\x1b: |\x1b \x1b: :\x1bG\x1b|\n| : | : : |\n| : : : : |\n| | : | : |\n|Y| : |B: |\n+---------+\n  (North)\n',
  'estado': 44,
  'ação': 1,
  'recompensa': -1},
 {'frame': '+---------+\n|\x1bR\x1b: | :\x1b \x1b:\x1bG\x1b|\n| : | : : |\n| : : : : |\n| | : | : |\n|Y| : |B: |\n+---------+\n  (East)\n',
  'estado': 64,
  'ação': 2,
  'recompensa': -1},
 {'frame': '+---------+\n|\x1bR\x1b: | : :\x1

In [25]:
from time import sleep
for frame in frames:
  clear_output(wait=True)
  print(frame['frame'])
  print('Estado', frame['estado'])
  print('Ação', frame['ação'])
  print('Recompensa', frame['recompensa'])
  sleep(0.1)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Estado 0
Ação 5
Recompensa 20
